In [4]:
%load_ext autoreload
%autoreload 2

import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"

import math
import argparse
import torch
import torch.utils.tensorboard
from torch.utils.data import DataLoader, TensorDataset, random_split
from torch.nn.utils import clip_grad_norm_
from tqdm.auto import tqdm

# from utils.dataset import *
from utils.misc import *
# from utils.data import *
# from models.vae_gaussian import *
# from models.vae_flow import *
# from models.flow import add_spectral_norm, spectral_norm_power_iteration
# from evaluation import *


from utils.utils_score import *
# from diffusionnet import *
from dataset.load_data_generated import *

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
# Arguments
THOUSAND=1000
parser = argparse.ArgumentParser()
# Model arguments
parser.add_argument('--model', type=str, default='gaussian', choices=['flow', 'gaussian'])
parser.add_argument('--num_steps', type=int, default=100)
parser.add_argument('--beta_1', type=float, default=1e-4)
parser.add_argument('--beta_T', type=float, default=1-1e-4)
parser.add_argument('--sched_mode', type=str, default='linear')

parser.add_argument('--residual', type=eval, default=True, choices=[True, False])#not_used
parser.add_argument('--latent_dim', type=int, default=128)
parser.add_argument('--layers', type=int, default=6)

# Optimizer and scheduler
parser.add_argument('--batch_size', type=int, default=128)
parser.add_argument('--max_grad_norm', type=float, default=10)
parser.add_argument('--lr', type=float, default=1e-3)
parser.add_argument('--end_lr', type=float, default=1e-5)
parser.add_argument('--sched_start_epoch', type=int, default=5*THOUSAND)
parser.add_argument('--sched_end_epoch', type=int, default=20*THOUSAND)
parser.add_argument('--max_iters', type=int, default=500*THOUSAND)

# Training
parser.add_argument('--seed', type=int, default=2020)
parser.add_argument('--device', type=str, default="cuda")

parser.add_argument('--model_tag', type=str, default='self_cross_v1')
parser.add_argument('--dataset', type=str, default='community_12_21_100')
parser.add_argument('--k', type=int, default=10)
parser.add_argument('--smallest', type=eval, default=False, choices=[True, False])

args = parser.parse_args([])
args.point_dim=args.k
args.use_mask=False
# seed_all(args.seed)


In [6]:
import sklearn
from sklearn import preprocessing
# datasetname='community_12_21_100'
# datasetname='planar_64_200'
# # # datasetname='sbm_200'
point_dimns = 10
train_set = LaplacianDatasetNX(args.dataset,'data/'+args.dataset,point_dim=args.k, smallest=args.smallest, split='train')



Point dim 10
Comp dimensions...
Comp stats...
Loaded precomuted eigenquantities
Comp Samples...
Tot #100


In [8]:
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=0,pin_memory=True)
# valid_loader = DataLoader(test_set, batch_size=len(test_set), shuffle=False, num_workers=0,pin_memory=False)


In [ ]:
from models.vae_gaussian import *
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint, EarlyStopping
from pytorch_lightning.loggers import WandbLogger

run_params=args

model = GaussianVAE(args)

# trainer = pl.Trainer(limit_train_batches=100, max_epochs=10,accelerator="auto")
checkpoint_callback = ModelCheckpoint(
    save_last=True,
    save_top_k=1,
    verbose=True,
    monitor='loss',
    mode='min'
)

# wandb_logger = WandbLogger(
#     name=f"{args.model_tag}_{args.k}",
#     project="graph_diffusion",
#     entity="l_cosmo"
# )
wandb_logger=None

trainer = pl.Trainer.from_argparse_args(
    run_params,
    callbacks=[checkpoint_callback],
    accelerator="auto",
    logger=wandb_logger,
    log_every_n_steps=250
)

trainer.fit(model, train_loader)


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [1]

  | Name      | Type           | Params
---------------------------------------------
0 | diffusion | DiffusionPoint | 5.6 M 
---------------------------------------------
5.6 M     Trainable params
0         Non-trainable params
5.6 M     Total params
22.345    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Epoch 0, global step 1: 'loss' reached 0.64302 (best 0.64302), saving model to '/home/lcosmo/PROJECTS/diffusion-graphs-perceptron/checkpoints/epoch=0-step=1.ckpt' as top 1
Epoch 1, global step 2: 'loss' reached 0.53699 (best 0.53699), saving model to '/home/lcosmo/PROJECTS/diffusion-graphs-perceptron/checkpoints/epoch=1-step=2.ckpt' as top 1
Epoch 2, global step 3: 'loss' reached 0.47706 (best 0.47706), saving model to '/home/lcosmo/PROJECTS/diffusion-graphs-perceptron/checkpoints/epoch=2-step=3.ckpt' as top 1
Epoch 3, global step 4: 'loss' was not in top 1
Epoch 4, global step 5: 'loss' was not in top 1
Epoch 5, global step 6: 'loss' reached 0.36252 (best 0.36252), saving model to '/home/lcosmo/PROJECTS/diffusion-graphs-perceptron/checkpoints/epoch=5-step=6.ckpt' as top 1
Epoch 6, global step 7: 'loss' was not in top 1
Epoch 7, global step 8: 'loss' was not in top 1
Epoch 8, global step 9: 'loss' was not in top 1
Epoch 9, global step 10: 'loss' was not in top 1
Epoch 10, global step 1

In [ ]:
# trainer.train_dataloader.dataset.datasets[0]
model.optimizers().param_groups[0]['lr']

In [ ]:
# # net_version = 'selfcross_v1'
# # ckpt_mgr = CheckpointManager('./experiments/'+datasetname+'_'+net_version+'/')
# ckpt = torch.load('../diffusion-graphs/experiments/community_12_21_100_selfcross_v1/ckpt_0.000000_390758.pt')
# model.load_state_dict(ckpt['state_dict'])

In [ ]:
import scipy

def sample_graphs(max_nodes, num_eigs,  num_graphs=1024, device='cuda'):
    # Sample eigenvectors and eigenvalues
    gen_pcs = []
    with torch.no_grad():
        x = model.sample(max_nodes, num_graphs, 1, point_dim=num_eigs, device=device)
        samples_EIGVEC = x.detach().cpu()

    # reconstruct laplacian matrix
    LLLall =[]
    for i,X in enumerate(samples_EIGVEC.cpu()):
        xx = X[:-1,:]
        yy = X[-1:,:]

        xx,yy = unscale_xy(xx,yy)
        yy=yy.float()

        xx_o=xx

        #orthogonal projection
        if False:
            U,s,V = torch.svd(xx)
            xx_o = (U@V).float()

        #discard samples that did not led to a quasi-orthonormal basis
        L = (xx_o*yy)@xx_o.t()  
        err = ((xx_o.t())@xx_o-torch.eye(xx.shape[-1])).norm()

        LLLall.append((err,L.numpy()))


    LLLall = [l[1] for l in sorted(LLLall, key=lambda e:e[0])][:]

    #reconstruct graph
    graph_pred_list = []
    for i,pp in enumerate(range(len(LLLall))):
        mynew = np.around(LLLall[pp],2)
        np.fill_diagonal(mynew,np.around(np.diag(mynew),0))
        mask = np.diag(mynew)>0

        siad = len(np.diag(mynew))
        adjb = np.zeros((siad,siad))

        for jj in range(len(np.diag(mynew))):
            if np.diag(mynew)[jj]!=np.diag(mynew)[jj]:
                break
            diagval = int(np.diag(mynew)[jj])

            row = mynew[jj,:]
            row[jj]=100
            idx =row.argsort()[:diagval]
            adjb[jj,idx] =1

        Arec = nx.Graph(adjb[mask,:][:,mask])
        graph_pred_list.append(Arec) 
        
    return graph_pred_list

    

In [ ]:
import scipy
from utils.eval_helper import degree_stats, clustering_stats, orbit_stats_all, eval_fraction_unique, eval_fraction_unique_non_isomorphic_valid, spectral_stats


# train_set = trainer.train_dataloader.dataset
# test_set = trainer.val_dataloaders.dataset


def evaluate(train_set, test_set):
    batch = next(iter(DataLoader(test_set, batch_size=len(test_set), shuffle=False, num_workers=0)))
    b,n,d = batch[0].shape
    
    graph_pred_list = sample_graphs(max_nodes=n, num_eigs=d,  num_graphs=1024, device='cuda')[:b]
    
    graph_pred_list_remove_empty = [G for G in graph_pred_list if not G.number_of_nodes() == 0]

    #compute metrics
    graph_test_list = []
    for jj in range(len(test_set)):
        laplacian_matrix = np.array(test_set[jj][3].cpu())[:test_set[jj][4],:test_set[jj][4]]
        Aori = np.copy(laplacian_matrix)
        np.fill_diagonal(Aori,0)
        Aori= Aori*(-1)
        graph_ref_list.append(nx.from_numpy_array(Aori)) 

    graph_train_list = []
    for jj in range(len(train_set)):
        laplacian_matrix = np.array(train_set[jj][3].cpu())[:train_set[jj][4],:train_set[jj][4]]
        Aori = np.copy(laplacian_matrix)
        np.fill_diagonal(Aori,0)
        Aori= Aori*(-1)
        graph_train_list.append(nx.from_numpy_array(Aori)) 


    degree, cluster, orbit, unique, novel, spectral = 0,0,0,0,0,0
    if len(graph_pred_list_remove_empty)>0:
        degree = degree_stats( graph_test_list,graph_pred_list_remove_empty, compute_emd=False)
        cluster = clustering_stats( graph_test_list,graph_pred_list_remove_empty, compute_emd=False)
        orbit = orbit_stats_all(graph_test_list, graph_pred_list_remove_empty, compute_emd=False)
        unique,novel,_ = eval_fraction_unique_non_isomorphic_valid(graph_pred_list_remove_empty,graph_train_list)
        spectral = spectral_stats(graph_test_list, graph_pred_list_remove_empty)
        
    return degree, cluster, orbit, unique, novel, spectral

evaluate(train_set, test_set)

In [ ]:
import scipy
LLLall =[]

for i,X in enumerate(samples_EIGVEC.cpu()):
    xx = X[:-1,:]
    yy = X[-1:,:]
    
    xx,yy = unscale_xy(xx,yy)
    yy=yy.float()
    
    xx_o=xx
    
    #orthogonal projection
    if False:
        U,s,V = torch.svd(xx)
        xx_o = (U@V).float()
    
    #discard samples that did not led to a quasi-orthonormal basis
    L = (xx_o*yy)@xx_o.t()  
    err = ((xx_o.t())@xx_o-torch.eye(xx.shape[-1])).norm()
    if err>300.0:
           continue

    LLLall.append((err,L.numpy()))
    
    if i<5:
        I =  torch.eye(xx.shape[0],device=xx.device)#.repeat(xx.shape[0],1,1)
        A = -(L - L*I)
    #     A = A - A.min(-1)[0].min(-1)[0]
    #     A = A/A.max(-1)[0].max(-1)[0] 
    #     A = A - A*I + I
        ZZ = (xx_o.t())@xx_o 
    #     print(ZZ)
    #     plt.figure()
        plt.imshow(ZZ)
        plt.colorbar()
        plt.show()


LLLall = [l[1] for l in sorted(LLLall, key=lambda e:e[0])][:20]
print(len(LLLall))

In [ ]:
# import scipy
# LLLall =[]


# point_dimns = 12
# dataset = LaplacianDatasetNX(datasetname,datasetname,point_dim=point_dimns, smallest=False)

# # for i,X in enumerate(samples_EIGVEC.cpu()):
# for i in range(len(dataset)):
    
#     xx = dataset[i][0].cpu()
#     yy = dataset[i][1].cpu().float()
#     print(xx.shape)
#     U,s,V = torch.svd(xx)
#     xx_o = (U@V).float()
#     L = (xx_o*yy)@xx_o.t()  
#     LLLall.append(L.numpy())
    
#     if i<5:
#         fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(8, 3))
        
#         I =  torch.eye(xx.shape[0],device=xx.device)#.repeat(xx.shape[0],1,1)
#         A = -(L - L*I)
#         ZZ = (xx.t())@xx 
#         axes[0].imshow(L)
        
#         axes[1].imshow(dataset[i][3].cpu())
        
        
# #         plt.colorbar()
#         plt.show()

In [ ]:
graph_pred_list = []

for i,pp in enumerate(range(len(LLLall))):
    mynew = np.around(LLLall[pp],2)
    
#     dg = np.diag(mynew)+0
    np.fill_diagonal(mynew,np.around(np.diag(mynew),0))
    
#     np.fill_diagonal(mynew,torch.diag(dataset[i][3]))
    
    
    
    mask = np.diag(mynew)>0

    siad = len(np.diag(mynew))
    adjb = np.zeros((siad,siad))

    for jj in range(len(np.diag(mynew))):
        if np.diag(mynew)[jj]!=np.diag(mynew)[jj]:
            break
        diagval = int(np.diag(mynew)[jj])

        row = mynew[jj,:]
        row[jj]=100
        idx =row.argsort()[:diagval]
        adjb[jj,idx] =1

    Arec = nx.Graph(adjb[mask,:][:,mask])
   
    graph_pred_list.append(Arec) 
    if i< 5:
#         print(dg)
#         print(mask.sum())
        plt.figure()
        nx.draw_kamada_kawai(Arec)

In [ ]:
graph_ref_list = []
for jj in range(len(test_set)):
    laplacian_matrix = np.array(dataset[jj][3].cpu())[:dataset[jj][4],:dataset[jj][4]]
    Aori = np.copy(laplacian_matrix)
    np.fill_diagonal(Aori,0)
    Aori= Aori*(-1)
    graph_ref_list.append(nx.from_numpy_array(Aori)) 
 
graph_ref_list_2 = []
for jj in range(len(train_set)):
    laplacian_matrix = np.array(dataset[jj][3].cpu())[:dataset[jj][4],:dataset[jj][4]]
    Aori = np.copy(laplacian_matrix)
    np.fill_diagonal(Aori,0)
    Aori= Aori*(-1)
    graph_ref_list.append(nx.from_numpy_array(Aori)) 

In [ ]:
print(len(graph_ref_list))

In [ ]:
#compare degree

import utils.evaluat_meas as mmd

# mmd_degree = degree_stats(graph_test, graphs)
sample_ref_deg  = []
sample_pred_deg  = []


for gr in graph_ref_list:
    degree_temp = np.array(nx.degree_histogram(gr))
    sample_ref_deg.append(degree_temp)
    
graph_pred_list_remove_empty = [G for G in graph_pred_list if not G.number_of_nodes() == 0]
for gp in graph_pred_list_remove_empty:
    degree_temp = np.array(nx.degree_histogram(gp))
    sample_pred_deg .append(degree_temp)
    
mmd_dist_deg = mmd.compute_mmd(sample_ref_deg , sample_pred_deg , kernel=mmd.gaussian_tv)
mmd_dist_deg 

In [ ]:
# clustering_stats
sample_ref_clus = []
sample_pred_clus = []

bins=100
for gr in graph_ref_list:
    clustering_coeffs_list = list(nx.clustering(gr).values())
    hist,_ = np.histogram(clustering_coeffs_list, bins=bins, range=(0.0, 1.0), density=False)
    sample_ref_clus.append(hist)
    

                                  
for gp in graph_pred_list_remove_empty:
    clustering_coeffs_list =  list(nx.clustering(gp).values())
    hist,_ = np.histogram(clustering_coeffs_list, bins=bins, range=(0.0, 1.0), density=False)
    sample_pred_clus.append(hist)     
    
mmd_dist_clus = mmd.compute_mmd(sample_ref_clus, sample_pred_clus, kernel=mmd.gaussian_tv, sigma=1.0 / 10)#, distance_scaling=bins)
mmd_dist_clus

In [ ]:
from utils.eval_helper import degree_stats, clustering_stats, orbit_stats_all, eval_fraction_unique, eval_fraction_unique_non_isomorphic_valid, spectral_stats

graph_ref_list = []
for jj in range(len(test_set)):
    laplacian_matrix = np.array(dataset[jj][3].cpu())[:dataset[jj][4],:dataset[jj][4]]
    Aori = np.copy(laplacian_matrix)
    np.fill_diagonal(Aori,0)
    Aori= Aori*(-1)
    graph_ref_list.append(nx.from_numpy_array(Aori)) 
    
degree = degree_stats( graph_ref_list,graph_pred_list_remove_empty, compute_emd=False)
cluster = clustering_stats( graph_ref_list,graph_pred_list_remove_empty, compute_emd=False)
orbit = orbit_stats_all(graph_ref_list_2, graph_ref_list, compute_emd=False)
unique,novel,_ = eval_fraction_unique_non_isomorphic_valid(graph_pred_list_remove_empty,graph_ref_list)
spectral = spectral_stats(graph_ref_list, graph_pred_list_remove_empty)

In [ ]:

#unique and novel
